#### http://35.193.70.70:8888/notebooks/TF/Probability/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/Chapter3_MCMC/Ch3_IntroMCMC_TFP.ipynb

##### The idea is to generate some data via Poisson distribution, with given lambdas. Then figure out what those lambdas are

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

lambda1Actual = 1.5
lambda2Actual = 4.2
# This is the "solution space" for Lambda1 (x_) and Lambda2 (y_)
gridX = gridY = np.linspace(.01, 5, 100)
alphaX = 0.3
alphaY = 0.1


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
# Specify Lambda and generate the data. These lambdas are what we're trying to figure out (pretending we don't know it)
def getData(lambda1Actual, lambda2Actual, N):
    dist   = tfd.Poisson(rate=lambda1Actual)
    xVals = dist.sample(sample_shape=(N, 1))
    dist   = tfd.Poisson(rate=lambda2Actual)
    yVals = dist.sample(sample_shape=(N, 1))
    data = tf.concat([xVals, yVals], axis=1)
    return tf.cast(data, dtype=tf.float64).eval()

In [4]:
# Model Lambdas with Exponential distribution, taking a guess at alphas
# This is just the initial prior: once we're in the loop then posterior becomes "prior"
def initPrior():
    rawX = tfd.Exponential(rate=alphaX).log_prob(tf.to_float(gridX))
    rawY = tfd.Exponential(rate=alphaY).log_prob(tf.to_float(gridY))
    exp_x_ = tf.reshape(rawX, shape=[-1,1])
    exp_y_ = tf.transpose(tf.reshape(rawY, shape=[-1,1]))
    pTheta  = tf.math.add(exp_x_,exp_y_)
    return pTheta.eval()

In [5]:
def plotPrior(prior):
    plt.figure(figsize(10, 6))
    plt.contour(gridX, gridY, prior)
    jet = plt.cm.jet
    im = plt.imshow(prior, interpolation='none', origin='lower',
                    cmap=jet, extent=(0, 5, 0, 5))
    plt.scatter(lambda1Actual, lambda2Actual, c="k", s=50, edgecolor="none")
    plt.xlim(0, 5)
    plt.ylim(0, 5)
    plt.title("Landscape formed by Exponential priors on $p_1, p_2$.")

In [6]:
def getLikelihood(x_, y_, X, Y):
    likelihoodX = tfd.Poisson(rate=x_).log_prob(X)
    likelihoodY = tfd.Poisson(rate=y_).log_prob(Y)
    return tf.math.add(likelihoodX, tf.transpose(likelihoodY))

In [7]:
def getPoint(numRows, numCols, prior):
    prior = prior.eval()
    Max = -np.inf
    idx = None
    for col in range(numCols):
        if prior[:,col].max() > Max:
            Max = prior[:,col].max()
            idx = col
    lambda1 = idx/numCols*5
    
    Max = -np.inf
    idx = None
    for row in range(numRows):
        if prior[row,:].max() > Max:
            Max = prior[row,:].max()
            idx = row
        lambda2 = idx/numRows*5
    return lambda1, lambda2

In [63]:
nList = [320]
points = []      # These are the best guesses of Lambda1 and 2 while looping over the data

for N in nList:
    with tf.Session() as sess:
        data = getData(lambda1Actual, lambda2Actual, N)
        prior = initPrior()
        numCols  = int(prior.shape[1])
        numRows = int(prior.shape[0])
        for n in range(N):
            x = data[n][0]
            y = data[n][1]
            x = np.reshape(x, newshape=[1,1])
            y = np.reshape(y, newshape=[1,1])
            l = getLikelihood(gridX, gridY, x, y)
            prior = l + prior
            if n%10 == 0:
                prior_ = sess.run(prior)
                bestL1, bestL2 = getPoint(numRows, numCols, prior)
                tup = (N, bestL1, bestL2)
                points.append(tup)
        bestL1, bestL2= getPoint(numRows, numCols, prior)
        tup = (N, bestL1, bestL2)
        points.append(tup)

In [64]:
print("Actuals")
actual = np.array([lambda1Actual, lambda2Actual])
print("- ","Lambda1: ", lambda1Actual)
print("- ","Lambda2: ", lambda2Actual, "\n")
print("' N'{:>10}{:>12}{:>12}".format("L1", "L2", "Error"))
for p in points:
    N, L1, L2 = p
    guess = np.array([L1,L2])
    err = np.linalg.norm(actual-guess)
    print("{:>3}{:>12.2f}{:>10.2f}{:>10.2f}".format(N,L1, L2,err))

Actuals
-  Lambda1:  1.5
-  Lambda2:  4.2 

' N'        L1          L2       Error
320        2.70      3.05      1.66
320        1.15      3.70      0.61
320        1.25      3.80      0.47
320        1.30      3.75      0.49
320        1.30      3.85      0.40
320        1.40      3.95      0.27
320        1.35      4.20      0.15
320        1.30      4.20      0.20
320        1.25      4.20      0.25
320        1.30      4.20      0.20
320        1.40      4.05      0.18
320        1.40      4.05      0.18
320        1.40      4.05      0.18
320        1.45      4.00      0.21
320        1.45      3.95      0.25
320        1.45      3.95      0.25
320        1.45      4.00      0.21
320        1.45      4.00      0.21
320        1.50      3.95      0.25
320        1.50      4.00      0.20
320        1.50      4.00      0.20
320        1.55      4.00      0.21
320        1.55      4.00      0.21
320        1.60      4.00      0.22
320        1.55      4.00      0.21
320        1.55  